In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.7 MB/s eta 0:00:00


In [ ]:
#library
import pandas as pd
import regex as re
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pickle
import plotly.express as px
from collections import defaultdict
from keras.models import load_model

## Dataset

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/binar-data-science/Platinum Challenge/train_preprocess.tsv.txt',sep='	',header=None)

In [ ]:
df.columns=['text','label']

In [ ]:
df.head()

,text,label
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


In [ ]:
df['text']

0        warung ini dimiliki oleh pengusaha pabrik tahu...
1        mohon ulama lurus dan k212 mmbri hujjah partai...
2        lokasi strategis di jalan sumatera bandung . t...
3        betapa bahagia nya diri ini saat unboxing pake...
4        duh . jadi mahasiswa jangan sombong dong . kas...
                               ...                        
10995                                         tidak kecewa
10996    enak rasa masakan nya apalagi kepiting yang me...
10997          hormati partai-partai yang telah berkoalisi
10998    pagi pagi di tol pasteur sudah macet parah , b...
10999    meskipun sering belanja ke yogya di riau junct...
Name: text, Length: 11000, dtype: object

In [ ]:
df.isna().sum()

text         0
label        0
text_baru    0
dtype: int64

In [ ]:
df.label.value_counts()

positive    6416
negative    3436
neutral     1148
Name: label, dtype: int64

## Cleansing

In [ ]:
df_stopwordbahasa=pd.read_csv('/content/drive/MyDrive/binar-data-science/Platinum Challenge/stopwordbahasa.csv',header=None)
df_stopwordbahasa.columns=['stopword']

In [ ]:
df_kamusalay = pd.read_csv('/content/drive/MyDrive/binar-data-science/Platinum Challenge/new_kamusalay.csv', encoding='latin-1', sep = ';', names=['Alay','Normal'])

In [ ]:
#cleansing
def preprocess_text(text):
    # lower text
    text = text.lower()

    # remove byte emoticons and special characters
    text = re.sub(r'\\x[\da-fA-F]{2}', '', text)

    # only allow characters from a to z (both upper and lower case)
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)

    # remove excessive white space
    text = re.sub(r'\s+', ' ', text)

    # remove digit or numbers
    text = re.sub(r"\b\d+\b", " ", text)

    # remove digit from string
    text = re.sub("\S*\d\S*", "", text).strip()

    return text

#removing stopwords
stopwords = df_stopwordbahasa['stopword'].tolist()
def remove_stopwords(text):
    list_stopwords = text.split()
    return ' '.join([text for text in list_stopwords if text not in stopwords])

#normalization
kamus_alay = dict(zip(df_kamusalay['Alay'], df_kamusalay['Normal']))
def normalize(text):
    for word in kamus_alay:
        return ' '.join([kamus_alay[word] if word in kamus_alay else word for word in text.split(' ')])

#stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()
def stem_text(text):
    return stemmer.stem(text)

In [ ]:
def cleansing_text(text):
    text = preprocess_text(text)
    text = remove_stopwords(text)
    text = normalize(text)
    text = stem_text(text)
    return text

In [ ]:
df['text_baru'] = df['text'].apply(lambda x: cleansing_text(x))

In [ ]:
cleaned_data = df.text_baru.to_list()

## Feature Extraction

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
count_vect.fit(cleaned_data)

X = count_vect.transform(cleaned_data)
print ("Feature Extraction selesai")

Feature Extraction selesai


In [ ]:
import pickle

pickle.dump(count_vect, open("feature.p", "wb"))

## Split Data

In [ ]:
from sklearn.model_selection import train_test_split

classes = df.label
classes

0        positive
1         neutral
2        positive
3        positive
4        negative
           ...   
10995    positive
10996    positive
10997     neutral
10998    negative
10999    positive
Name: label, Length: 11000, dtype: object

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, classes, test_size = 0.2)

## Training

In [ ]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier()
model.fit(X_train, y_train)

print ("Training selesai")

Training selesai


In [ ]:
#simpan model
pickle.dump(model, open("model.p", "wb"))

## Evaluation

In [ ]:
from sklearn.metrics import classification_report

test = model.predict(X_test)

print ("Testing selesai")
print(classification_report(y_test, test))

Testing selesai
              precision    recall  f1-score   support

    negative       0.70      0.70      0.70       710
     neutral       0.67      0.59      0.63       236
    positive       0.84      0.85      0.84      1254

    accuracy                           0.78      2200
   macro avg       0.74      0.71      0.72      2200
weighted avg       0.77      0.78      0.77      2200



## Cross Validation

In [ ]:
kf = KFold(n_splits=5,random_state=42,shuffle=True)

accuracies = []

y = classes

for iteration, data in enumerate(kf.split(X), start=1):

    data_train   = X[data[0]]
    target_train = y[data[0]]

    data_test    = X[data[1]]
    target_test  = y[data[1]]

    clf = MLPClassifier()
    clf.fit(data_train,target_train)

    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test,preds)

    print("Training ke-", iteration)
    print(classification_report(target_test,preds))
    print("======================================================")

    accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)

print()
print()
print()
print("Rata-rata Accuracy: ", average_accuracy)

Training ke- 1
              precision    recall  f1-score   support

    negative       0.70      0.70      0.70       680
     neutral       0.70      0.53      0.60       239
    positive       0.83      0.87      0.85      1281

    accuracy                           0.78      2200
   macro avg       0.74      0.70      0.72      2200
weighted avg       0.78      0.78      0.78      2200

Training ke- 2
              precision    recall  f1-score   support

    negative       0.70      0.72      0.71       706
     neutral       0.62      0.59      0.60       220
    positive       0.86      0.85      0.86      1274

    accuracy                           0.78      2200
   macro avg       0.73      0.72      0.72      2200
weighted avg       0.78      0.78      0.78      2200

Training ke- 3
              precision    recall  f1-score   support

    negative       0.70      0.70      0.70       682
     neutral       0.71      0.62      0.66       215
    positive       0.85      0

## Predict

In [ ]:
original_text =  '''
syukur
'''

text = count_vect.transform([cleansing_text(original_text)])

result = model.predict(text)[0]
print("Sentiment:")
print()
print(result)

Sentiment:

positive


In [ ]:
original_text =  '''
sombong
'''

text = count_vect.transform([cleansing_text(original_text)])

result = model.predict(text)[0]
print("Sentiment:")
print()
print(result)

Sentiment:

negative
